# A Large Scale Fish Dataset

In this notebook, we are going to create a neural network for classifying the pictures with their species.

The data is retrived from [A Large-Scale Dataset for Fish Segmentation and Classification](https://www.kaggle.com/crowww/a-large-scale-fish-dataset)



In [2]:
!pip install jovian --upgrade --quiet
!pip install opendatasets --upgrade --quiet

In [ ]:
# Execute this to save new versions of the notebook
jovian.commit(project="zerotogans-project")

In [1]:
import jovian
import opendatasets as od
import os
import numpy as np 
import pandas as pd 
from IPython.display import display
#pd.options.display.max_columns = None
#pd.options.display.max_rows = None
#pd.reset_option("max_rows")
pd.options.display.max_colwidth = 300


import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
%matplotlib inline


from PIL import Image
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

<IPython.core.display.Javascript object>

At the beginning, we will first download the data by using the 'opendatasets' packages

In [2]:
dataset_url = 'https://www.kaggle.com/crowww/a-large-scale-fish-dataset'
od.download(dataset_url)

a-large-scale-fish-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


Now we are going to explore the instance of this dataset.
As we will see below, the dataset contains 9 species and there are two folders containing the images of each species and some Readme documents. One of the folder contains the RGB type picture and another one contains black and white types picture. In this notebook, we will only focus on the RGB type pictures.

In [3]:
DATA_DIR = './a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset'
print(os.listdir(DATA_DIR))

['Black Sea Sprat', 'Gilt-Head Bream', 'Hourse Mackerel', 'license.txt', 'README.txt', 'Red Mullet', 'Red Sea Bream', 'Sea Bass', 'Segmentation_example_script.m', 'Shrimp', 'Striped Red Mullet', 'Trout']


Now we will use this feature to create our list of species

In [4]:
species = []

for directory in os.listdir(DATA_DIR):
    if "." not in directory:           # Removes all documents that is not a folder
        species.append(directory)
        
species

['Black Sea Sprat',
 'Gilt-Head Bream',
 'Hourse Mackerel',
 'Red Mullet',
 'Red Sea Bream',
 'Sea Bass',
 'Shrimp',
 'Striped Red Mullet',
 'Trout']

Here we will create a list of paths to link up the images and the labels.

In [5]:
image_paths = []
image_classes = np.array([[cls]*1000 for cls in species]).reshape(-1) # create a classes array of shape (9000,)

for spc in species:
    # adds the path of the image to the first column
    image_paths.extend(os.path.join(DATA_DIR, spc, spc, i) for i in os.listdir(os.path.join(DATA_DIR, spc, spc))) 

In [6]:
image_paths

['./a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset\\Black Sea Sprat\\Black Sea Sprat\\00001.png',
 './a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset\\Black Sea Sprat\\Black Sea Sprat\\00002.png',
 './a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset\\Black Sea Sprat\\Black Sea Sprat\\00003.png',
 './a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset\\Black Sea Sprat\\Black Sea Sprat\\00004.png',
 './a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset\\Black Sea Sprat\\Black Sea Sprat\\00005.png',
 './a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset\\Black Sea Sprat\\Black Sea Sprat\\00006.png',
 './a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset\\Black Sea Sprat\\Black Sea Sprat\\00007.png',
 './a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset\\Black Sea Sprat\\Black Sea Sprat\\00008.png',
 './a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset\\Black Sea Sprat\\Black Sea Sprat\\00009.png',
 './a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset\\Black 

In [7]:
data = pd.DataFrame({'path':image_paths, 'class':image_classes, })
data

,path,class
0,./a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset\Black Sea Sprat\Black Sea Sprat\00001.png,Black Sea Sprat
1,./a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset\Black Sea Sprat\Black Sea Sprat\00002.png,Black Sea Sprat
2,./a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset\Black Sea Sprat\Black Sea Sprat\00003.png,Black Sea Sprat
3,./a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset\Black Sea Sprat\Black Sea Sprat\00004.png,Black Sea Sprat
4,./a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset\Black Sea Sprat\Black Sea Sprat\00005.png,Black Sea Sprat
...,...,...
8995,./a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset\Trout\Trout\00996.png,Trout
8996,./a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset\Trout\Trout\00997.png,Trout
8997,./a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset\Trout\Trout\00998.png,Trout
8998,./a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset\Trout\Trout\00999.png,Trout


Then we will apply some transformation to the pictures
- resize to 256 x 256. this is because the original solution is 2832 x 2128 or 1024 x 768. That is too large for our studying 
- Normalized to [-1,1]

In [8]:
transform = transforms.Compose([
        transforms.Resize((256,256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5,0.5,0.5),std=(0.5,0.5,0.5))
    ])

We will now create a dictionary for labelling the species by integer.

In [9]:
str_to_int = {
'Red Mullet' : 0 ,
 'Striped Red Mullet' : 1,
 'Sea Bass':2,
 'Gilt-Head Bream':3,
 'Shrimp':4,
 'Red Sea Bream':5,
 'Hourse Mackerel':6,
 'Trout':7,
 'Black Sea Sprat':8
}

Now, we define the class of dataset that use for showing the pictures

In [10]:
class MarketFishDataset(Dataset):
    def __init__(self, data, root_dir, transform=transforms.ToTensor()):
        self.data = data
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        img_name = self.data.iloc[idx, 0]
        image = Image.open(img_name)
        y_label = torch.tensor(str_to_int[self.data.iloc[idx, 1]])
        
        if self.transform:
            image = self.transform(image)
    
        return (image, y_label)

In [11]:
dataset = MarketFishDataset(
    data=data,
    root_dir=DATA_DIR,
    transform=transform
)

In [12]:
img, lab = dataset[42]
img.numpy().transpose((1, 2, 0))

array([[[-1., -1., -1.],
        [-1., -1., -1.],
        [-1., -1., -1.],
        ...,
        [-1., -1., -1.],
        [-1., -1., -1.],
        [-1., -1., -1.]],

       [[-1., -1., -1.],
        [-1., -1., -1.],
        [-1., -1., -1.],
        ...,
        [-1., -1., -1.],
        [-1., -1., -1.],
        [-1., -1., -1.]],

       [[-1., -1., -1.],
        [-1., -1., -1.],
        [-1., -1., -1.],
        ...,
        [-1., -1., -1.],
        [-1., -1., -1.],
        [-1., -1., -1.]],

       ...,

       [[-1., -1., -1.],
        [-1., -1., -1.],
        [-1., -1., -1.],
        ...,
        [-1., -1., -1.],
        [-1., -1., -1.],
        [-1., -1., -1.]],

       [[-1., -1., -1.],
        [-1., -1., -1.],
        [-1., -1., -1.],
        ...,
        [-1., -1., -1.],
        [-1., -1., -1.],
        [-1., -1., -1.]],

       [[-1., -1., -1.],
        [-1., -1., -1.],
        [-1., -1., -1.],
        ...,
        [-1., -1., -1.],
        [-1., -1., -1.],
        [-1., -1., -1.]]

In [ ]:
img, lab = dataset[5000]
plt.imshow(img.numpy().transpose((1, 2, 0)))
plt.title(species[int(lab)])
plt.axis('off')
plt.show()

In [13]:
batch_size = 50
torch.manual_seed(100)
train_set, test_set = torch.utils.data.random_split(dataset, [7500,1500])
train_loader = DataLoader(dataset=train_set,batch_size=batch_size,shuffle=True)
test_loader = DataLoader(dataset=test_set,batch_size=batch_size,shuffle=True)

In [14]:
len(train_loader)

150

In [15]:
a = next(iter(train_loader))
a[0].size()

torch.Size([50, 3, 256, 256])

Here we will create the neural network instance

In [16]:
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [21]:
class CNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(CNN, self).__init__()
        self.output_dim = output_dim
        
        self.convs = nn.Sequential(
            nn.Conv2d(input_dim, 6, 16, stride=6, padding=0), #the first 6 is number of feature maps, the second 16 is the kernel size
            nn.ReLU(),
            nn.Conv2d(6, 12, 5, stride=2, padding=0),
            nn.ReLU(),
            nn.Conv2d(12, 24, 7, stride=2, padding=0),
            nn.ReLU()
        )
        
        self.q1 = nn.Linear(7*7*24, 32) # the final size can be calculated by [(original size-kernel size) /stride] +1
        self.q2 = nn.Linear(32, output_dim)
    
    def forward(self, x):
        conv = self.convs(x)
        flat = conv.reshape(-1, 7*7*24)
        q1 = F.relu(self.q1(flat))
        q = self.q2(q1)
        return q

In [18]:
#dataset[4999][0].shape

In [22]:
input_dim = dataset[0][0].shape[0] # Number of channels in the image (3)
output_dim = len(species) # Number of classes (9)

loss_nn = nn.CrossEntropyLoss()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CNN(input_dim, output_dim).to(device)

learning_rate = 3e-4
optimizer_nn = torch.optim.Adam(model.parameters(), lr=learning_rate)

now we will train our model

In [23]:
epochs = 10
iterations = 0
loss_list = []
iteration_list = []
accuracy_list = []

for epoch in range(epochs):
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        train = Variable(images.view(batch_size, 3, 256, 256))
        labels = Variable(labels)
        
        outputs = model(train)
        loss = loss_nn(outputs, labels)
        
        predictions = torch.max(outputs, 1)[1].to(device)
    
        optimizer_nn.zero_grad()
        
        loss.backward()
        
        optimizer_nn.step()
        
        iterations += 1
        
        if iterations % 50 == 0:
            corrects = 0
            total = 0
            
            for images, labels in test_loader:

                images, labels = images.to(device), labels.to(device)

                test = Variable(images.view(batch_size, 3, 256, 256))

                outputs = model(test)

                predict = torch.max(outputs.data, 1)[1].to(device)

                total += len(labels)

                corrects += (predict == labels).sum()
        
            acuracy = 100 * corrects / float(total)

            loss_list.append(loss.data)
            iteration_list.append(iterations)

    print(f"Epoch: {epoch+1} | Loss: {loss.data} | Accuracy: {acuracy}")

Epoch: 1 | Loss: 1.3484951257705688 | Accuracy: 54.20000076293945
Epoch: 2 | Loss: 1.1533839702606201 | Accuracy: 66.46666717529297
Epoch: 3 | Loss: 0.6251171827316284 | Accuracy: 73.86666870117188
Epoch: 4 | Loss: 0.46960148215293884 | Accuracy: 79.13333129882812
Epoch: 5 | Loss: 0.3210476338863373 | Accuracy: 82.4000015258789
Epoch: 6 | Loss: 0.4834093153476715 | Accuracy: 86.46666717529297
Epoch: 7 | Loss: 0.27964043617248535 | Accuracy: 87.33333587646484
Epoch: 8 | Loss: 0.2885204553604126 | Accuracy: 87.86666870117188
Epoch: 9 | Loss: 0.16058506071567535 | Accuracy: 89.86666870117188
Epoch: 10 | Loss: 0.08814725279808044 | Accuracy: 90.93333435058594


In [29]:
torch.save(model.state_dict(), 'zerotogans-project-conv2d-7x7.pth')

In [20]:
#torch.load('zerotogans-project.pth')

OrderedDict([('convs.0.weight',
              tensor([[[[ 0.0345,  0.0249,  0.0914,  ...,  0.0008,  0.0153,  0.1038],
                        [ 0.0880,  0.0935,  0.0168,  ..., -0.0175,  0.0478,  0.0586],
                        [ 0.0026,  0.0680,  0.0293,  ...,  0.0052,  0.0456,  0.0895],
                        ...,
                        [-0.0077,  0.0472, -0.0004,  ...,  0.0210,  0.0887,  0.0182],
                        [ 0.1025,  0.0659,  0.1136,  ...,  0.0184,  0.0210,  0.0457],
                        [-0.0244,  0.0915,  0.1064,  ...,  0.0094,  0.0641,  0.1103]],
              
                       [[ 0.0702, -0.0334, -0.0562,  ...,  0.0211, -0.0250,  0.0129],
                        [ 0.0682,  0.0182, -0.0067,  ...,  0.0338, -0.0473,  0.0065],
                        [ 0.0762, -0.0007,  0.0058,  ...,  0.0164, -0.0563, -0.0786],
                        ...,
                        [-0.0457,  0.0336,  0.0327,  ..., -0.0188, -0.0521, -0.0104],
                        [-0.0596, 

plot the accuracy vs iteration

In [ ]:
plt.figure(figsize=(25,6))
plt.plot(iteration_list,loss_list)
plt.xlabel("Number of Iteration")
plt.ylabel("Loss")
plt.title("CNN: Loss vs Number of Iteration")
plt.show()

now we will check a random picture with its prediction

In [25]:
def prediction(index, data):
    img, _ = data[index]
    img = img.to(device)
    img = Variable(img.view([-1, 3, 224, 224]))
    outputs = model(img)
    predict = torch.max(outputs.data, 1)[1].to(device)
    
    return classes[int(predict)]

In [ ]:
index = 1536

fig = plt.figure(figsize=(10, 7))

img, label = dataset[index]
predicted_label = prediction(index, dataset)

fig.add_subplot(1, 2, 1)
plt.imshow(img.numpy().transpose((1, 2, 0)))
plt.axis('off')
plt.title("Real Label: " + classes[int(label)])

fig.add_subplot(1,2,2)
plt.imshow(img.numpy().transpose((1, 2, 0)))
plt.axis('off')
plt.title("Predicted Label: " + predicted_label)

plt.show()

In [27]:
jovian.log_hyperparams(arch='conv2d-7*7', 
                       lrs=3e-4,
                       Loss=0.08814725279808044,
                       Accuracy=90.93333435058594,
                       epochs=10)

[jovian] Hyperparams logged.


In [30]:
jovian.commit(project='zerotogans-project', outputs=['zerotogans-project-conv2d-7x7.pth'], environment=None)

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Updating notebook "arthasli001/zerotogans-project" on https://jovian.ai/
[jovian] Uploading notebook..
[jovian] Uploading additional outputs...
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ai/arthasli001/zerotogans-project


'https://jovian.ai/arthasli001/zerotogans-project'

Final conclusion:
- we uploaded two version of neural network architecture
    - version 5: 3 layer of convolution network(8,5,6 kernel size) with 2 full-connected network (full version)
        - Loss: 0.01663968339562416 | Accuracy: 95.4000015258789
    - version 7: 3 layer of convolution network(16,5,7 kernel size) with 2 full-connected network (simple version)
        - Loss: 0.08814725279808044 | Accuracy: 90.93333435058594
        
    - as we predicted, the simple version is lower in accuracy but it is faster to run 
    - we can use this dataset and the trained model as an application to further classify the fish species in the sea